In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer
import urllib.request
import warnings 
warnings.simplefilter('ignore')
import plotly.express as px
%matplotlib inline

In [30]:
# reading Taz_Yeshuv_250_mapping
def reading_town_code_to_taz_mapper():
    town_code_to_taz_mapper =  pd.read_excel(r'/Users/yotamdery/Old Desktop/git/health_data_mapping/Taz_Yeshuv_250_mapping.xlsx',engine= 'openpyxl', header= 3, usecols= 'A, Q:AF')
    #town_code_to_taz_mapper.columns = ['taz_id','town_code','population']
    town_code_to_taz_mapper = town_code_to_taz_mapper.iloc[:2630]
    return town_code_to_taz_mapper

In [38]:
taz_ages_pop = reading_town_code_to_taz_mapper()

In [39]:
# Merging columns to match the SEIR model's age-groups:
taz_ages_pop['10-19'] = taz_ages_pop['10-14'] + taz_ages_pop['age_19-15']
taz_ages_pop['20-29'] = taz_ages_pop['20-24'] + taz_ages_pop['25-29']
taz_ages_pop['30-39'] = taz_ages_pop['30-34'] + taz_ages_pop['35-39']
taz_ages_pop['40-49'] = taz_ages_pop['40-44'] + taz_ages_pop['45-49']
taz_ages_pop['50-59'] = taz_ages_pop['50-54'] + taz_ages_pop['55-59']
taz_ages_pop['60-69'] = taz_ages_pop['60-64'] + taz_ages_pop['65-69']
taz_ages_pop['70+'] = taz_ages_pop['70-74'] + taz_ages_pop['75+']

In [42]:
# Dropping old columns and renaming for 'taz'
taz_ages_pop.drop(columns= taz_ages_pop.columns[3:17], axis= 1, inplace= True)
taz_ages_pop.rename(columns= {'Unnamed: 0' : 'taz'}, inplace= True)

In [50]:
# Adding 'population' prefix for the age-group columns:
new_columns_names_list = ['taz']
for column in taz_ages_pop.columns[1:]:
    new_columns_names_list.append('population_' + column)
taz_ages_pop.columns = new_columns_names_list

In [51]:
taz_ages_pop

,taz,population_0-4,population_5-9,population_10-19,population_20-29,population_30-39,population_40-49,population_50-59,population_60-69,population_70+
0,1,389.0,413.0,1180.0,719.0,269.0,373.0,419.0,178.0,68.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,108.0,92.0,120.0,94.0,161.0,162.0,118.0,167.0,152.0
...,...,...,...,...,...,...,...,...,...,...
2625,2626,2086.0,1739.0,2920.0,2094.0,1159.0,889.0,435.0,233.0,143.0
2626,2627,1196.0,1069.0,1953.0,1359.0,794.0,647.0,286.0,125.0,113.0
2627,2628,2437.0,2118.0,3834.0,2739.0,1559.0,1211.0,583.0,280.0,185.0
2628,2629,1937.0,1899.0,3262.0,3365.0,3375.0,3309.0,2547.0,1774.0,1136.0


In [54]:
# reading taz2county_id file
taz2county_id =  pd.read_excel(r'/Users/yotamdery/Old Desktop/git/SEIR_model_covid_yotams copy/Data/division_choice/30/taz2county_id.xlsx',engine= 'openpyxl')

In [56]:
taz_county_df = taz_ages_pop.merge(town_code_to_taz_mapper, how= 'inner', left_on= 'taz', right_on= 'taz_id')

In [64]:
taz_county_df

,taz,population_0-4,population_5-9,population_10-19,population_20-29,population_30-39,population_40-49,population_50-59,population_60-69,population_70+,taz_id,cell_id
0,1,389.0,413.0,1180.0,719.0,269.0,373.0,419.0,178.0,68.0,1,4400
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5103
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,5103
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,5103
4,5,108.0,92.0,120.0,94.0,161.0,162.0,118.0,167.0,152.0,5,5103
...,...,...,...,...,...,...,...,...,...,...,...,...
2625,2626,2086.0,1739.0,2920.0,2094.0,1159.0,889.0,435.0,233.0,143.0,2626,6202
2626,2627,1196.0,1069.0,1953.0,1359.0,794.0,647.0,286.0,125.0,113.0,2627,6202
2627,2628,2437.0,2118.0,3834.0,2739.0,1559.0,1211.0,583.0,280.0,185.0,2628,6202
2628,2629,1937.0,1899.0,3262.0,3365.0,3375.0,3309.0,2547.0,1774.0,1136.0,2629,6200


In [70]:
res = taz_county_df.groupby('cell_id', as_index= False).agg(sum).drop('taz_id', axis= 1)

In [77]:
res

,county_id,population_0-4,population_5-9,population_10-19,population_20-29,population_30-39,population_40-49,population_50-59,population_60-69,population_70+
0,1100,85139.0,79836.0,138688.0,117076.0,93046.0,75600.0,58960.0,47663.0,46878.0
1,1101,37139.0,30580.0,53891.0,42732.0,26275.0,17149.0,16116.0,14252.0,15041.0
2,1103,19994.0,17726.0,25505.0,16342.0,13271.0,9463.0,5871.0,3662.0,2833.0
3,2100,13371.0,12516.0,22347.0,20757.0,15726.0,14556.0,13054.0,10756.0,9193.0
4,2200,9129.0,8358.0,15664.0,14385.0,10916.0,10472.0,9461.0,8364.0,6938.0
5,2202,2025.0,2073.0,4305.0,4362.0,3339.0,2725.0,2037.0,1081.0,738.0
6,2300,30301.0,29715.0,55679.0,48173.0,40838.0,41819.0,34445.0,29335.0,25062.0
7,2302,14838.0,14951.0,31764.0,26555.0,18582.0,18109.0,14029.0,7591.0,5414.0
8,2400,24348.0,24776.0,47636.0,42047.0,37360.0,38446.0,32883.0,28709.0,24009.0
9,2402,32992.0,33352.0,71575.0,58996.0,44137.0,42033.0,30856.0,15720.0,11332.0


In [73]:
res.rename(columns= {'cell_id' : 'county_id'}, inplace= True)

In [76]:
res.to_csv('population_per_county_age-group.csv', index= False)